In [3]:
%load_ext autoreload
%autoreload 2
import pickle
import iisignature as iisig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from utils.midi import *
from utils.data import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
with open('./data/dataframes/min_note_0_min_gap_0/dfs.pkl', 'rb') as f:
    dfs = pickle.load(f)

In [ ]:
note_dur_dfs = note_duration_transform(dfs, 0.1)

In [5]:
iisig.sig(dfs[0].values, 5)

RuntimeError: data must be (convertable to) a numpy array

In [6]:
dfs[0].values

array([[0.0, 0.49180300000000005, 75, 80, 'melody'],
       [0.49180300000000005, 0.9836060000000001, 73, 80, 'melody'],
       [0.9836060000000001, 1.4754090000000002, 71, 80, 'melody'],
       [1.4754090000000002, 1.9672120000000002, 68, 80, 'melody'],
       [1.9672120000000002, 2.4590150000000004, 71, 80, 'melody'],
       [2.4590150000000004, 2.9508180000000004, 78, 80, 'melody'],
       [2.9508180000000004, 3.4426210000000004, 75, 80, 'melody'],
       [3.4426210000000004, 3.9344240000000004, 73, 80, 'melody'],
       [3.9344240000000004, 4.426227000000001, 75, 80, 'melody'],
       [4.426227000000001, 4.918030000000001, 73, 80, 'melody'],
       [4.918030000000001, 5.409833000000001, 71, 80, 'melody'],
       [5.409833000000001, 5.901636000000001, 68, 80, 'melody'],
       [5.901636000000001, 6.393439000000001, 71, 80, 'melody'],
       [6.393439000000001, 6.885242000000001, 78, 80, 'melody'],
       [6.885242000000001, 7.377045000000001, 75, 80, 'melody'],
       [7.37704500000

In [ ]:
signatures = []
for df in dfs:
    signatures.append(iisig.sig(df.values, 5))